In [1]:
import numpy as np
import time

def new_f(x):
    time.sleep(0.1)
    return x * np.cos(x)

# Derivative Free Methods (continued)
In the last lecture, we derived a derivative free method called section search for finding the minimum of a function.  We found that section search was a little slower than the derivative based method (Newton's method) but fairly reliable.  (Reliability, in this context, means that section search takes the same number of steps no matter what function we are minimizing, whereas Newton's method can take drastically more steps for some functions.)  In this lecture, we are going to make some improvements to section search so that it runs faster.  

Until now, we have been measuring the "speed" of our iterative methods by checking how many iterations they require.  To be a bit more precise (and do give us an idea for how to improve the method), let's actually time our code and see how long it takes to minimize $f(x) = x\cos(x)$.  

The following is our code from last lecture with a timer around the main portion of the algorithm: 

In [3]:
f = lambda x: x * np.cos(x)
a = 2
b = 5
c = 0.5001
tolerance = 1e-8

t0 = time.time()
for k in range(1000):
    x = c * a + (1 - c) * b
    y = (1 - c) * a + c * b
    
    if f(x) < f(y):
        b = y
    else:
        a = x
    
    if (b - a) < tolerance:
        break
t1 = time.time()
print("Time elapsed: ", t1 - t0, "seconds")

Time elapsed:  0.0002257823944091797 seconds


The exact times will vary from computer to computer and from run to run, but on my machine this takes roughly 0.0002 seconds to run.  This is so fast that you might wonder why we are even bothering improving our code.  We know that section search will always take the same number of steps, regardless of the function we choose, so shouldn't it always take roughly the same amount of time?  (The number of steps does depend on $a$ and $b$, as well as the tolerance and our choice of $c$, but not on the function itself.)  The problem is that when we change our function we might also change how long it takes to run each step.  To demonstrate, I have written another function called `new_f` that also calculates $x\cos(x)$, but does so with a different (and much worse) method.  It is defined at the top of this notebook.  Let's try the section search method again with this new function.  

In [4]:
a = 2
b = 5
c = 0.5001
tolerance = 1e-8

t0 = time.time()
for k in range(1000):
    x = c * a + (1 - c) * b
    y = (1 - c) * a + c * b
    
    if new_f(x) < new_f(y):
        b = y
    else:
        a = x
    
    if (b - a) < tolerance:
        break
t1 = time.time()
print("Time elapsed: ", t1 - t0, "seconds")

Time elapsed:  5.8105385303497314 seconds


This time our code took roughly 5.8 seconds to run - several orders of magnitude slower!  The problem is that `new_f` is very slow.  If you peek at the definition (at the top of this notebook) you can see why: `new_f` pauses for 0.1 seconds before returning.  This explains why it took 5.8 seconds.  Every function call takes 0.1 seconds; the section search code takes 29 iterations; each iteration calls the function twice (in the first if statement).  This means we expect a total time of roughly $0.1\cdot 29 \cdot 2 = 5.8$ seconds.  

This is a common issue in scientific computing: We wrote code that is supposed to minimize any function (or at least any continuous function of one variable), so we cannot safely assume that we get to write the function or that the function will be fast.  In particular, it is possible for the function call `f(x)` to be much slower than everything else in our algorithm put together.  This might be because the function is badly written (as is arguably the case here), but more often such a situation arises because `f` is simply doing a lot of work.  For instance, if the function `f` solves a $1,000,000 \times 1,000,000$ system of equations, then it will take a very long time even if coded efficiently.  It therefore behooves us to try to call `f` as few times as possible.  This is a common theme in numerical algorithms.  You often save more time by reducing function calls than by reducing iterations.  

Of course, every time we go through the loop in our algorithm, we call `f` twice (once for $x$ and once for $y$), so reducing the number of iterations does reduce the number of function calls.  However, we could do even better if we rewrote our method so that it only had to call `f` once every time through the loop.  To see why this might be plausible, let's look at the $a$, $b$, $x$ and $y$ values that we use at two successive steps.  We will let $a_1 < x_1 < y_1 < b_1$ be the values in the first step and $a_2 < x_2 < y_2 < b_2$ be the values in the second step.  To make the analysis a little easier, let's assume that $f(x_1) < f(y_1)$.  (The same argument works with some $x$'s and $y$'s reversed if $f(y_1) < f(x_1)$, but we would then have to do twice as much algebra.)  According to the rules we derived before, $a_1$ and $a_2$ will be exactly the same and $b_2 = y_1$.  However, $x_2$ and $y_2$ have no relationship to the old values $x_1$ and $y_1$.  This is wasteful because we already spent time calculating $f(x_1)$ and $f(y_1)$.  If we reused these numbers then we wouldn't have to call $f$ again.  Of course, we have already used $y_1$ as our new $b$, so we can't also use it as a new guess, but $x_1$ is still in between $a_2$ and $b_2$, so it would be nice if there was a way to use it as one of our new guesses.  

There are two possibilities: We could use $x_1$ as our new guess for $x$ and make $x_2 = x_1$ or we could use $x_1$ as our new guess for $y$ and make $y_2 = x_1$.  Without working through some algebra, it is not obvious which of these is better.  It turns out that only the latter choice works, so we will just go through the math for that case, but you should try to copy this analysis with the choice $x_2 = x_1$ and see why it fails.  

We will therefore insist that $y_2 = x_1$.  How do we know that this is possible?  Remember that we derived the following formulas in the last class: 

$x_1 = ca_1 + (1 - c)b_1$, 

$y_1 = (1 - c)a_1 + cb_1$, 

$x_2 = ca_2 + (1 - c)b_2$, 

$y_2 = (1 - c)a_2 + cb_2$.  

(These are the same formulas as before, but with the subscripts one and two added.)  Let's take the formula for $y_2$ and try to simplify it.  We know that $a_2 = a_1$ and we know that $b_2 = y_1 = (1 - c)a_1 + cb_1$, so we get 

$y_2 = (1 - c)a_1 + c((1 - c)a_1 + cb_1)$.  

We are trying to make $y_2 = x_1$, so we have 

$x_1 = (1 - c)a_1 + ca_1 - c^2a_1 + c^2b_1$.  

Furthermore, we know the original formula for $x_1$, so we get 

$ca_1 + (1 - c)b_1 = (1 - c)a_1 + ca_1 - c^2a_1 + c^2b_1$.  

We can rearrange and factor this to get 

$(c^2 + c - 1)(b_1 - a_1) = 0$.  

This means that either $(b_1 - a_1) = 0$ or $c^2 + c - 1 = 0$.  We have already assumed that $a < b$, so the former case can't be true.  Therefore, we know that $c^2 + c - 1 = 0$, which means that 

$c = \frac{-1 \pm\sqrt{5}}{2}$.  

We have 

$c = \frac{-1 + \sqrt{5}}{2} \approx 0.618$ or $c = \frac{-1 - \sqrt{5}}{2} \approx -1.618$.

We already know that $c$ has to be between $0.5$ and $1$, so the latter case will not work, but the former is a perfectly valid choice of $c$.  The number $0.618$ is called the golden ratio (or more accurately, the golden ratio conjugate), and so when we use this $c$ value in the section search we call the method *golden section search*.  

Through a stroke of luck, it turns out that exactly the same analysis works in the case when $f(y_1) < f(x_1)$.  In that case, it turns out that we need to use our old guess for $y$ as the new guess for $x$, so we need to set $x_2 = y_1$.  If you go through the same algebra, you will find that this requires the same value of $c$.  

Let's try to modify our old code to use this new technique.  

In [5]:
f = lambda x: x * np.cos(x)
a = 2
b = 5

The first change we have to make is to use the new value of $c$.  

`c = (-1 + np.sqrt(5)) / 2
tolerance = 1e-8`

`for k in range(1000):
    x = c * a + (1 - c) * b
    y = (1 - c) * a + c * b
    if f(x) < f(y):
        b = y`
 
Next, if $f(x) < f(y)$ we want to make our new guess for $y$ equal to the old guess for $x$:

`       y = x
    else:
        a = x`

And if $f(y) < f(x)$ then we want to make our new guess for $x$ equal to the old guess for $y$:

`       x = y
    if (b - a) < tolerance:
        break`

If you try this code, you will find that it works just as well as before (except that it takes 41 steps instead of 29 because we used a larger $c$).  There are a couple of issues, which we will fix in turn.  

First, although we successfully reuse our guesses, we then immediately recalculate them at the top of the for loop.  To fix this, we should move all of our guesses into the two different cases of the if statement.  We would like to write code like this: 

`f = lambda x: x * np.cos(x)
a = 2
b = 5
c = (-1 + np.sqrt(5)) / 2
tolerance = 1e-8`

`for k in range(1000):
    if f(x) < f(y):
        b = y
        y = x
        x = c * a + (1 - c) * b
    else:
        a = x
        x = y
        y = (1 - c) * a + c * b`
    
`   if (b - a) < tolerance:
        break`
        
This way we would always make a guess for both $x$ and $y$ without doing any work twice.  The problem with this is that the first time we go through the loop we haven't yet defined $x$ or $y$, so our code will fail at the first if statement.  To fix this, we need to make our first guess for $x$ and $y$ before the loop starts.  

In [6]:
f = lambda x: x * np.cos(x)
a = 2
b = 5
c = (-1 + np.sqrt(5)) / 2
tolerance = 1e-8

x = c * a + (1 - c) * b
y = (1 - c) * a + c * b
for k in range(1000):
    if f(x) < f(y):
        b = y
        y = x
        x = c * a + (1 - c) * b
    else:
        a = x
        x = y
        y = (1 - c) * a + c * b
        
    if (b - a) < tolerance:
        break

Our code is now working again, but it still doesn't go any faster than before.  Remember that our whole goal was to reuse function values.  Right now, we are calling `f` twice in the if statement, but python just throws away the values of $f(x)$ and $f(y)$ after that line.  To reuse these values, we have to save them into variables.  Our strategy will be to calculate $f(x)$ or $f(y)$ and store them in variables as soon as we make a new $x$ or $y$.  In particular, let's call the variable holding $f(x)$ `fx` and let's call the variable holding $f(y)$ `fy`.  We get 

In [8]:
f = lambda x: x * np.cos(x)
a = 2
b = 5
c = (-1 + np.sqrt(5)) / 2
tolerance = 1e-8

x = c * a + (1 - c) * b
fx = f(x)
y = (1 - c) * a + c * b
fy = f(y)
for k in range(1000):
    if f(x) < f(y):
        b = y
        y = x
        fy = fx
        x = c * a + (1 - c) * b
        fx = f(x)
    else:
        a = x
        x = y
        fx = fy
        y = (1 - c) * a + c * b
        fy = f(y)
        
    if (b - a) < tolerance:
        break

Notice that when we reused a guess we also got to reuse its f value.  That is, we didn't have to call the function `f` again.  Finally, we can replace the function calls in our if statement with the values that we saved.  

In [11]:
f = lambda x: x * np.cos(x)
a = 2
b = 5
c = (-1 + np.sqrt(5)) / 2
tolerance = 1e-8

x = c * a + (1 - c) * b
fx = f(x)
y = (1 - c) * a + c * b
fy = f(y)
for k in range(1000):
    if fx < fy:
        b = y
        y = x
        fy = fx
        x = c * a + (1 - c) * b
        fx = f(x)
    else:
        a = x
        x = y
        fx = fy
        y = (1 - c) * a + c * b
        fy = f(y)
        
    if (b - a) < tolerance:
        break

If you try this code, you will see that it still finds the same minimum in the same number of steps, but now we only call the function `f` once per step (plus two calls before we begin the loop) instead of twice per step.  

If we try the same code using the slow function `new_f` then we can see how much time we actually saved: 

In [12]:
f = lambda x: x * np.cos(x)
a = 2
b = 5
c = (-1 + np.sqrt(5)) / 2
tolerance = 1e-8

t0 = time.time()
x = c * a + (1 - c) * b
fx = new_f(x)
y = (1 - c) * a + c * b
fy = new_f(y)
for k in range(1000):
    if fx < fy:
        b = y
        y = x
        fy = fx
        x = c * a + (1 - c) * b
        fx = new_f(x)
    else:
        a = x
        x = y
        fx = fy
        y = (1 - c) * a + c * b
        fy = new_f(y)
        
    if (b - a) < tolerance:
        break
t1 = time.time()
print("Time elapsed: ", t1 - t0, "seconds")

Time elapsed:  4.308884143829346 seconds


This code only took $4.3$ seconds instead of $5.8$.  This is easy to predict from our code.  The loop ran 41 times, and each step requires one function call.  We also had to call the function twice at the beginning, for a total of 43 calls.  Each call takes $0.1$ seconds, for a total time of $4.3$ seconds.  Remember tha the amount of time saved depends on how slow `new_f` is.  If the function calls each took an hour, then we would have saved 15 hours instead of 1.5 seconds.  

# Builtin Minimization Functions
Python has two optimization functions that you need to be aware of.  They are both in the `optimize` package, which is itself a subpackage of `scipy`.  We will have to import this package before we can use either function: 

In [13]:
import scipy.optimize

The first function is called `minimize_scalar`.  As the name suggests, this finds the minimum of a scalar function (i.e., a function that takes one number as input and returns one number as output).  The general syntax is 

`scipy.optimize.minimize_scalar(f, bounds=(a, b), method='Bounded')`

where `f` is the function you are trying to minimize and you are looking for a miniumum in the interval $[a, b]$.  The `bounds=(a,b)` part is mandatory - python will issue an error if you forget it.  The `method='Bounded'` part is not, but it is important.  The default method does not check if the solution is between $a$ and $b$, and so you will frequently get an answer outside of the bounds you specified.  

As an example, to find the minimum of $f(x) = x\cos(x)$ between $x = 2$ and $x = 5$, we would use the code

In [21]:
f = lambda x: x * np.cos(x)
xmin = scipy.optimize.minimize_scalar(f, bounds=(2, 5), method='Bounded')

The variable xmin has a somewhat unusual type: 

In [22]:
print(type(xmin))

<class 'scipy.optimize.optimize.OptimizeResult'>


Variables of the type `OptimizeResult` have several different properties, but the one we care the most about is `x`, which is the minimum we were looking for.  

In [23]:
print(xmin.x)

3.4256177072339398


The terminology here can be a little bit confusing.  The name `xmin` was entirely up to us, but the property name `x` is always the same.  For instance, we could have done this: 

In [25]:
this_is_a_silly_variable_name = scipy.optimize.minimize_scalar(f, bounds=(2, 5), method='Bounded')

but we still have to use `.x` to access the solution:

In [26]:
print(this_is_a_silly_variable_name.x)

3.4256177072339398


This function is essentially the gold standard for minimizing functions of one variable, but in practice that is often a pretty severe limitation.  It is very common to want to minimize functions of more than one variable.  For example, you might want to find the minimum of 

$f(x, y) = (x - 1)^2 + 3(y + 2)^2$.  

There is another builtin minimization function that can handle this case: `minimize`.  The general syntax is 

`scipy.optimize.minimize(f, x0, method='Nelder-Mead')`

where `f` is the function you are trying to minimize and `x0` is an initial guess that should be close to the minimum you are looking for.  As with `minimize_scalar`, the method is optional, but important.  If you don't specify `method='Nelder-Mead'` then python will use a derivative based method and need to approximate the derivative (using techniques we will learn in a couple of weeks).  

Unfortunately, the function `f` requires a somewhat awkward syntax.  The obvious way to define the above function in python would be to write 

In [27]:
f = lambda x, y: (x - 1) ** 2 + 3 * (y + 2) ** 2

You could then call the function by writing, for example, 

In [28]:
f(5, 7)

259

This plugs in 5 for $x$ and 7 for $y$ and gives the answer $(5 - 1)^2 + 3\cdot (7 + 2)^2 = 259$.  Unfortunately, `minimize` will not work with this anonymous function.  For example, if you wanted to use the initial guess $x = 5$, $y = 7$ then you might try something like 

`scipy.optimize.minimize(f, 5, 7, method='Nelder-Mead')`

or

`scipy.optimize.minimize(f, (5, 7), method='Nelder-Mead')`

but these would just produce errors.  

The problem is that `minimize` expects a function with only one argument.  This sounds problematic, because $f$ is a function of two variables, not just one.  The trick is that function arguments don't have to just be single numbers.  We will instead write a function that takes an *array* as an argument.  Let's call the array `v`.  You should think of `v` as being the array `[x, y]`, but you never actually define this in python.  Instead, if you need to use x then you refer to it as `v[0]`, which is the first entry of `v`.  Likewise, if you need to use `y` then you refer to it as `v[1]`, which is the second entry of `v`.  We will therefore define our function as 

In [30]:
f = lambda v: (v[0] - 1) ** 2 + 3 * (v[1] + 2) ** 2

Calling this function is a little bit more complicated than before.  If you try to use `f(5, 7)` as we did before, then python will give you an error because `f` is only supposed to take one argument.  Instead, you have to give it an array with both the `x` and `y` values you want to plug in, like so: 

In [31]:
f(np.array([5, 7]))

259

(Actually, the `np.array()` part is unnecessary, and you could also just write `f([5, 7])` or even `f((5, 7))`.  Both are ok, but for this class it will be easier to stick with arrays.)  

We can now use `minimize` to find the minimum of $f$.  In this example, I used an initial guess of $x = 5$ and $y = 7$.  The actual minimum is at $x = 1$ and $y = -2$, so we get an answer very close to `[1, -2]`.  

In [34]:
xmin = scipy.optimize.minimize(f, np.array([5, 7]), method='Nelder-Mead')

As with `minimize_scalar`, the output `xmin` has the type `OptimizeResult`.  If we want to use the answer, we need to write `xmin.x`.  

In [35]:
print(xmin.x)

[ 1.00001619 -1.99997925]


It is important to remember that `minimize`, much like Newton's method, is highly dependent on the initial guess.  If you guess too far from the minimum you are looking for, then it is entirely possible for `minimize` to find a completely different minimum or for `minimize` to not find any answer at all.  

It is also worth noting (although probably obvious from the names) that both `minimize_scalar` and `minimize` are designed to find minima, not maxima.  There is no builtin function in `scipy` that is designed to find the maximum of an arbitrary function.  If you want to find maxima, you need to use the same trick that we discussed for section search: If you are looking for the maximum of $f(x)$, you can instead find the minimum of $-f(x)$.  So, for example, if yuo wanted to find the maximum of 

$f(x, y) = -(x - 2)^2 - (y - 3)^2$

with an initial guess of $x = 1$ and $y = 2$, then you could use the code

In [36]:
f = lambda v: -(-(v[0] - 2) ** 2 - (v[1] - 3) ** 2)
xmin = scipy.optimize.minimize(f, np.array([1, 2]), method='Nelder-Mead')
print(xmin.x)

[2.00003595 2.99995609]


The extra negative signs in f can become confusing, and it is usually not very convenient to have different functions that just differ by a sign, so another equivalent approach would be to define the negative version when you call `minimize` and never actually give it a name.  The following code finds exactly the same answer as the previous version: 

In [37]:
f = lambda v: -(v[0] - 2) ** 2 - (v[1] - 3) ** 2
xmin = scipy.optimize.minimize(lambda v: -f(v), np.array([1, 2]), method='Nelder-Mead')
print(xmin.x)

[2.00003595 2.99995609]
